In [12]:
import pandas as pd
#df = pd.read_csv('../../top_rated_wines.csv')
df = pd.read_csv('../../../organization.csv')
#df = df[df['variety'].notna()] # remove any NaN values as it blows up serialization
df = df[df['Reports To'].notna()] # remove any NaN values as it blows up serialization
df = df[df['Line Detail 1'].notna()] # remove any NaN values as it blows up serialization
df = df[df['Line Detail 2'].notna()] # remove any NaN values as it blows up serialization
#df = df[df['Organization Name'].notna()] # remove any NaN values as it blows up serialization
data = df.to_dict('records')
#data = df.sample(700).to_dict('records') # Get only 700 records. More records will make it slower to index
len(data)

243

In [13]:
from qdrant_client import models, QdrantClient
from sentence_transformers import SentenceTransformer

In [14]:
encoder = SentenceTransformer('all-MiniLM-L6-v2') # Model to create embeddings

In [15]:
# create the vector database client
qdrant = QdrantClient(":memory:") # Create in-memory Qdrant instance

In [16]:
# Create collection to store wines
qdrant.recreate_collection(
    #collection_name="top_wines",
    collection_name="org_chart",
    vectors_config=models.VectorParams(
        size=encoder.get_sentence_embedding_dimension(), # Vector size is defined by used model
        distance=models.Distance.COSINE
    )
)

/var/folders/db/h74khb5x6_jfk0g2t1d1ptsw0000gp/T/ipykernel_86189/3813498731.py:2: DeprecationWarning: `recreate_collection` method is deprecated and will be removed in the future. Use `collection_exists` to check collection existence and `create_collection` instead.
  qdrant.recreate_collection(


True

In [28]:
# vectorize!
qdrant.upload_points(
    collection_name="org_chart", # change "top_wines" to "org_chart" if using organization data
    points=[
        models.PointStruct(
            id=idx,
            vector=encoder.encode(doc["Line Detail 1"]).tolist(), # change "notes" to "Line Detail 2" if using organization data
            payload=doc,
        ) for idx, doc in enumerate(data) # data is the variable holding all the wines
    ]
)

In [29]:
#user_prompt = "Suggest me an amazing Malbec wine from Argentina"
user_prompt = "Suggest me an amazing R&D Manager from Bangalore"

In [30]:
# Search time for awesome wines!

hits = qdrant.search(
    collection_name="org_chart", # change "top_wines" to "org_chart" if using organization data
    query_vector=encoder.encode(user_prompt).tolist(),
    limit=10
)
for hit in hits:
  print(hit.payload, "score:", hit.score)

{'Unique Identifier': '10_Naresh_Medisetty_(1045807)', 'Name': 'Naresh Medisetty (1045807)', 'Reports To': '1_Cathal_Copas_(1043184)', 'Line Detail 1': 'Manager, R&D', 'Line Detail 2': 'Karnataka, Bangalore #78, 7th Floor'} score: 0.6128553560894914
{'Unique Identifier': '45_Rahul_Gupta_(1497183)', 'Name': 'Rahul Gupta (1497183)', 'Reports To': '7_Vijay_Indoria_(1045867)', 'Line Detail 1': 'Manager, R&D', 'Line Detail 2': 'Karnataka, Bangalore #78, 7th Floor'} score: 0.6128553560894914
{'Unique Identifier': '20_Partha_Nayak_(1045812)', 'Name': 'Partha Nayak (1045812)', 'Reports To': '2_Chris_Gerardot_(1041034)', 'Line Detail 1': 'Manager, R&D', 'Line Detail 2': 'Karnataka, Bangalore #78, 7th Floor'} score: 0.6128553560894914
{'Unique Identifier': '43_Anand_Molkeri_(1045810)', 'Name': 'Anand Molkeri (1045810)', 'Reports To': '7_Vijay_Indoria_(1045867)', 'Line Detail 1': 'Manager, R&D', 'Line Detail 2': 'Karnataka, Bangalore #78, 7th Floor'} score: 0.6128553560894914
{'Unique Identifier'

/var/folders/db/h74khb5x6_jfk0g2t1d1ptsw0000gp/T/ipykernel_86189/2256241894.py:3: DeprecationWarning: `search` method is deprecated and will be removed in the future. Use `query_points` instead.
  hits = qdrant.search(


In [31]:
# define a variable to hold the search results
search_results = [hit.payload for hit in hits]

In [32]:
# Now time to connect to the local large language model - Execute the repos/llama/llava-v1.5-7b-q4.llamafile
from openai import OpenAI
client = OpenAI(
    base_url="http://127.0.0.1:8080/v1", # "http://<Your api-server IP>:port"
    api_key = "sk-no-key-required"
)
completion = client.chat.completions.create(
    model="LLaMA_CPP",
   # messages=[
   #     {"role": "system", "content": "You are chatbot, a wine specialist. Your top priority is to help guide users into selecting amazing wine and guide them with their requests."},
   #     {"role": "user", "content": "Suggest me an amazing Malbec wine from Argentina"},
   #     {"role": "assistant", "content": str(search_results)}
   # ]
    messages=[
        {"role": "system", "content": "You are chatbot, a HR specialist. Your top priority is to help guide users into selecting amazing talent and guide them with their requests."},
        {"role": "user", "content": "Suggest me an amazing R&D Manager from Bangalore"},
        {"role": "assistant", "content": str(search_results)}
    ]
)
print(completion.choices[0].message)

ChatCompletionMessage(content='Here are five amazing R&D Manager candidates from Bangalore:\n\n1. Naresh Medisetty (1045807)\n2. Rahul Gupta (1497183)\n3. Partha Nayak (1045812)\n4. Anand Molkeri (1045810)\n5. Vijay Indoria (1045867)\n\nThese candidates have a strong background in R&D and have been praised for their exceptional work by their previous supervisors. They have experience working in various R&D roles and have been recognized for their innovative ideas and contributions to their teams.\n\nPlease note that these candidates are not currently open positions, but they may be suitable for future opportunities. If you have any specific requirements or would like more information, please let me know.</s>', refusal=None, role='assistant', annotations=None, audio=None, function_call=None, tool_calls=None)
